In [ ]:
# Next step!
# Use fstrings
# Use data classes to store data
# try to install KITES

In [1]:
import numpy as np
import fabio
from datetime import datetime
from numba import jit, cuda, prange
import time
import math

In [16]:
# initialze detector info
# //These are the details of the detector in the detInfo wave
# 	//detInfo[0] = xdim (pixels)
# 	//detInfo[1] = ydim (pixels)
# 	//detInfo[2] = xpixsize	(mm)
# 	//detInfo[3] = ypixsize	(mm)
# 	//detInfo[4] = xcen (pixels)
# 	//detInfo[5] = ycen (pixels)
# 	//detInfo[6] = tiltang	Given in degrees
# 	//detInfo[7] = tiltrot		Given in degrees
# 	//detInfo[8] = sample to detector distance	Given in mm
# 	//detInfo[9] = lamda	Given in Angstroms
# detInfo = np.array([1475,1679,0.17200001,0.17200001,738.00598,829.4032,0,0,450,0.23946001])
detInfo = np.array([2048,2048,0.085,0.085,1062.08,1042.1,-0.655953,133.021,714.029,0.16118])
imFilePrefix = ""
imNumbers = np.arange(1,1000)
boxsize = 501
qmax = 5
# boxname = ""
omegaPos = np.arange(9.8,-50.2,-0.2)
phiPos = np.zeros_like(omegaPos)
psiPos = np.zeros_like(omegaPos)
mask = []
rejects = []
cust_mask = []
mask_list = []
rotmatrix = np.array([0.61190927,0.098435618,-0.78477865,-0.78786123,-0.011431515,-0.61574668,0.069582619,-0.99507779,-0.070558563]).reshape((3,3))
normInt = np.arange(1,1000)
startIm = 0
# box = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
rotmatrix

array([[ 0.61190927,  0.09843562, -0.78477865],
       [-0.78786123, -0.01143151, -0.61574668],
       [ 0.06958262, -0.99507779, -0.07055856]])

In [23]:
def DiffuseReconstruction(fileNum):
    # t0 = time.perf_counter()
    tiltang = detInfo[6]*np.pi/180; tiltrot = detInfo[7]*np.pi/180; dist = detInfo[8]
    
#     row,col = np.indices((int(detInfo[0]),int(detInfo[1])))
    row = np.arange(0,int(detInfo[0])).repeat(int(detInfo[1])).reshape(int(detInfo[0]),int(detInfo[1]))
    col = np.arange(0,int(detInfo[1])).repeat(int(detInfo[0])).reshape(int(detInfo[1]),int(detInfo[0]))
    col = np.transpose(col)
#     TT = (np.arctan(np.sqrt((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    TT = (np.sqrt(np.arctan((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    AZ = np.arctan2(((col-detInfo[5])*detInfo[3]), ((row-detInfo[4])*detInfo[2]))+np.pi 

    # use floats for bigbox intensities and unsigned integers for number of pixels
    bigbox = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
    numpixels = np.zeros((boxsize,boxsize,boxsize),dtype=np.uint32)
    # Use ravel() for shallow copy or flatten() for deep copy

    qrangex = np.linspace(-(qmax),qmax,boxsize)
    qrangey = np.linspace(-(qmax),qmax,boxsize)
    qrangez = np.linspace(-(qmax),qmax,boxsize)

    # Make/copy arrays to gpu device
    d_TT = cuda.to_device(TT)
    d_AZ = cuda.to_device(AZ)
    d_qx = cuda.device_array_like(d_TT)
    d_qy = cuda.device_array_like(d_TT)
    d_qz = cuda.device_array_like(d_TT)
    d_rotmatrix = cuda.to_device(rotmatrix)
    d_rotated_qx = cuda.device_array_like(d_TT)
    d_rotated_qy = cuda.device_array_like(d_TT)
    d_rotated_qz = cuda.device_array_like(d_TT)
    d_BBposX = cuda.device_array_like(d_TT)
    d_BBposY = cuda.device_array_like(d_TT)
    d_BBposZ = cuda.device_array_like(d_TT)
    #set the output file from the CUDA kernel
    d_bigboxlooper = cuda.device_array_like(bigbox)
    d_numpixelslooper = cuda.device_array_like(numpixels)
    #pin the memory for the image to be loaded into the CUDA kernel
    pinned_CI = cuda.pinned_array(TT.shape)

    qrangexstart = qrangex[0]
    qrangexdelta = qrangex[1]
    qrangeystart = qrangey[0]
    qrangeydelta = qrangey[1]
    qrangezstart = qrangez[0]
    qrangezdelta = qrangez[1]
    wavelength = detInfo[9]
    detector_xdim = detInfo[0]
    detector_ydim = detInfo[1]

    # CUDA variables
    # The number of threads per block should be a round multiple of the warp size, which is 32 on all current hardware.
    blocks = (128,128)
    threads_per_block = (24,24)
    # blocks = (64,64)
    # threads_per_block = (16,16)

    for i in range(fileNum):
        # if i % 10 == 0:
        # print(i)
        #use FABIO to load the image
        CI_fabio = fabio.open("C:\\Users\\Patrick\\Desktop\\s1_363K_2p0_diffuse\\DisCorr\\s1_363K_2p0_diffuse_"+("%04d" % (i))+".edf")
        pinned_CI = CI_fabio.data
        
        omega = omegaPos[i]*np.pi/180; phi = phiPos[i]*np.pi/180; psi = psiPos[i]*np.pi/180

        d_CI = cuda.to_device(pinned_CI)
        # d_CI = cuda.to_device(CI)

        looper[blocks,threads_per_block](detector_xdim, detector_ydim, omega, phi, psi, wavelength, qrangexstart, qrangexdelta, qrangeystart, qrangeydelta, qrangezstart, qrangezdelta, d_TT, d_AZ, d_CI, d_qx, d_qy, d_qz, d_rotmatrix, d_rotated_qx, d_rotated_qy, d_rotated_qz, d_BBposX, d_BBposY, d_BBposZ, d_bigboxlooper, d_numpixelslooper)

    bigbox = d_bigboxlooper.copy_to_host()
    numpixels = d_numpixelslooper.copy_to_host()

    #ignore dividing by inf or 0
    np.seterr(divide='ignore', invalid='ignore')
    bigbox /= numpixels
    
    return bigbox, numpixels

In [24]:
@cuda.jit
def looper(detector_xdim, detector_ydim, omega, phi, psi, wavelength, qrangexstart, qrangexdelta, qrangeystart, qrangeydelta, qrangezstart, qrangezdelta, TT, AZ, CI, qx, qy, qz, rotmatrix, rotated_qx, rotated_qy, rotated_qz, BBposX, BBposY, BBposZ, bigboxlooper, numpixelslooper):
    grid_row, grid_col = cuda.grid(2)
    stride_row, stride_col = cuda.gridsize(2)
    
    for data_row in range(grid_row,detector_xdim,stride_row):
        for data_col in range(grid_col,detector_ydim,stride_col):

            qx[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (math.sin(TT[data_row][data_col])*(math.sin(phi)*math.sin(psi)*math.sin(omega) + math.cos(phi)*math.cos(omega)) + math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.sin(phi)*math.cos(psi) - math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*(math.sin(phi)*math.sin(psi)*math.cos(omega) - math.cos(phi)*math.sin(omega)))
            qy[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (-math.sin(TT[data_row][data_col])*(math.cos(phi)*math.sin(psi)*math.sin(omega) - math.sin(phi)*math.cos(omega)) - math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.cos(phi)*math.cos(psi) + math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*(math.cos(phi)*math.sin(psi)*math.cos(omega) + math.sin(phi)*math.sin(omega)))
            qz[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (math.sin(TT[data_row][data_col])*math.cos(psi)*math.sin(omega) - math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*math.cos(psi)*math.cos(omega) - math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.sin(psi))

            rotated_qx[data_row][data_col] = rotmatrix[0][0] * qx[data_row][data_col] + rotmatrix[0][1] * qy[data_row][data_col] + rotmatrix[0][2] * qz[data_row][data_col]
            rotated_qy[data_row][data_col] = rotmatrix[1][0] * qx[data_row][data_col] + rotmatrix[1][1] * qy[data_row][data_col] + rotmatrix[1][2] * qz[data_row][data_col]
            rotated_qz[data_row][data_col] = rotmatrix[2][0] * qx[data_row][data_col] + rotmatrix[2][1] * qy[data_row][data_col] + rotmatrix[2][2] * qz[data_row][data_col]

            BBposX[data_row][data_col] = round((rotated_qx[data_row][data_col] - (qrangexstart)) / (qrangexdelta-qrangexstart))
            BBposY[data_row][data_col] = round((rotated_qy[data_row][data_col] - (qrangeystart)) / (qrangeydelta-qrangeystart))
            BBposZ[data_row][data_col] = round((rotated_qz[data_row][data_col] - (qrangezstart)) / (qrangezdelta-qrangezstart))
        
            if(0 <= BBposX[data_row][data_col] and 0 <= BBposY[data_row][data_col] and 0 <= BBposZ[data_row][data_col] and BBposX[data_row][data_col] < bigboxlooper.shape[0] and BBposY[data_row][data_col] < bigboxlooper.shape[1] and BBposZ[data_row][data_col] < bigboxlooper.shape[2] ):
                cuda.atomic.add(bigboxlooper, (BBposX[data_row][data_col], BBposY[data_row][data_col], BBposZ[data_row][data_col]), CI[data_row][data_col])
                cuda.atomic.add(numpixelslooper, (BBposX[data_row][data_col], BBposY[data_row][data_col], BBposZ[data_row][data_col]), 1)

                
                    # if(np.isfinite(CI[j])):

    

In [25]:
import gc
#garbage collector
gc.collect()

startTime = datetime.now()
box,num = DiffuseReconstruction(10)
print(datetime.now() - startTime)

0:00:06.783495


In [6]:
from matplotlib import pyplot       # Load matplotlib
%matplotlib qt
pyplot.imshow(box[:, :, 250]) 
pyplot.show() 


<IPython.core.display.Javascript object>

In [26]:
box[250, 250, 260]

nan

In [7]:
import winsound
winsound.Beep(300, 750)

In [27]:
%reload_ext line_profiler

%lprun -f looper

Timer unit: 1e-07 s

<function numba.cuda.api.list_devices()>

In [27]:
import numba
numba.cuda.is_available()

True

In [27]:

from numba import cuda

@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [6]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [29]:
test = np.linspace(1,9,9).reshape(3,3)
blah = np.zeros_like(test)

In [30]:
matmul(rotmatrix, test, blah)

In [31]:
print(blah)

[[-4.48779881  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
